In [1]:
from ast import literal_eval
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/kexinhuang/Downloads/TDC Leaderboard Submission Form 6.csv')

In [2]:
df = df.rename(columns = {
    df.columns.values[0]: 'Time',
    df.columns.values[1]: 'Name',
    df.columns.values[2]: 'Email',
    df.columns.values[3]: 'Model',
    df.columns.values[4]: 'Paper_URL',
    df.columns.values[5]: 'Code_URL',
    df.columns.values[6]: 'Hardware',
    df.columns.values[7]: '#Params',
    df.columns.values[8]: 'Group',
    df.columns.values[9]: 'Text_Result',
    df.columns.values[10]: 'PKL_Result',    
})
df = df.drop(df.columns.values[11], axis = 1)

In [3]:
def return_text(x):
    try:
        y = literal_eval(x)
    except:
        y = x
    return y
df['Result'] = df.Text_Result.apply(lambda x: return_text(x))

In [4]:
metrics_direction = {
    'mae': 'desc',
    'roc-auc': 'asc', 
    'pr-auc': 'asc',
    'pcc': 'asc'
}

In [5]:
def retrieve_mean(x, categories):
    l = []
    for i in categories:
        l.append(x[i][0])
    return np.array(l)

## ADMET Group

In [39]:
df_admet = df[df.Group == 'ADMET Group']

In [40]:
admet_metrics = {'caco2_wang': 'mae',
            'hia_hou': 'roc-auc',
            'pgp_broccatelli': 'roc-auc', 
            'bioavailability_ma': 'roc-auc',
            'lipophilicity_astrazeneca': 'mae',
            'solubility_aqsoldb': 'mae',
            'bbb_martins': 'roc-auc',
            'ppbr_az': 'mae',
            'vdss_lombardo': 'pcc',
            'cyp2c9_veith': 'pr-auc',
            'cyp2d6_veith': 'pr-auc',
            'cyp3a4_veith': 'pr-auc',
            'cyp2c9_substrate_carbonmangels': 'pr-auc',
            'cyp3a4_substrate_carbonmangels': 'roc-auc',
            'cyp2d6_substrate_carbonmangels': 'pr-auc',
            'half_life_obach': 'pcc',
            'clearance_hepatocyte_az': 'pcc',
            'clearance_microsome_az': 'pcc',
            'ld50_zhu': 'mae',
            'herg': 'roc-auc',
            'ames': 'roc-auc',
            'dili': 'roc-auc'
            }

In [41]:
ADMET = {
    'Absorption': ['caco2_wang', 'hia_hou', 'pgp_broccatelli', 'bioavailability_ma', 'lipophilicity_astrazeneca', 'solubility_aqsoldb'],
    'Distribution': ['bbb_martins', 'ppbr_az', 'vdss_lombardo'],
    'Metabolism': ['cyp2c9_veith', 'cyp2d6_veith', 'cyp3a4_veith', 'cyp2c9_substrate_carbonmangels', 'cyp2d6_substrate_carbonmangels', 'cyp3a4_substrate_carbonmangels'],
    'Excretion': ['half_life_obach', 'clearance_hepatocyte_az', 'clearance_microsome_az'],
    'Toxicity': ['ld50_zhu', 'herg', 'ames', 'dili']
}

In [43]:
def get_entries(bm):
    results = df_admet.Result.apply(lambda x: x[bm][0]).values
    print(results)
    
    '''
    rank_dict = dict(zip(range(results.shape[0]), list([0] * results.shape[0])))
    
    for idx, i in enumerate(range(results.T.shape[0])):
        # for every benchmark
        sort_idx = np.argsort(results.T[i], axis = 0)
        if metrics_direction[admet_metrics[ADMET[category][idx]]] == 'desc':
            sort_idx = np.argsort(results.T[i], axis = 0)
        else:
            sort_idx = np.argsort(results.T[i], axis = 0)[::-1]
        
        for pos, val in enumerate(sort_idx):
            rank_dict[val] = rank_dict[val] + pos
    
    rank = [] # [rank for 1, rank for 2, ...]
    for i in range(results.shape[0]):
        rank.append(rank_dict[i])
        
    out = np.argsort(rank)
    
    idx2rank = dict(zip(out, list(range(len(out)))))
    rank2idx = dict(zip(list(range(len(out))), out))
    '''
    out = np.argsort(results)
    idx2rank = dict(zip(out, list(range(len(out)))))
    rank2idx = dict(zip(list(range(len(out))), out))
    
    print('---- ' + bm + ' ------')
    print('---- ' + admet_metrics[bm] + ' ------')
    entries = []
    for submit_idx in range(len(df_admet)):
        temp = df_admet.iloc[submit_idx]
        html_code = '<tr> \n'
        html_code = html_code + '  <td> ' + str(idx2rank[submit_idx] + 1) + ' </td> \n'        
        html_code = html_code + '  <td> ' + temp.Model + ' </td> \n'
        html_code = html_code + '  <td><a href="mailto:' + temp.Email + '">' + temp.Name + '</a></td> \n'
        html_code = html_code + '  <td><a href="' + temp.Code_URL + '"> GitHub</a>, <a href="' + temp.Paper_URL + '">Paper </a></td> \n'
        value = temp['#Params']
        html_code = html_code + '  <td> ' + f'{value:,}' + ' </td> \n'
        html_code = html_code + '  <td>' + format(temp.Result[bm][0], '.3f') + ' <span>&#177;</span> ' + format(temp.Result[bm][1], '.3f') +' </td> \n'
        html_code += '</tr>'

        entries.append(html_code)
    
    for i in range(len(out)):
        print(entries[rank2idx[i]])

In [44]:
for i, j in ADMET.items():
    for x in j:
        get_entries(x)

[0.446 0.393 0.908]
---- caco2_wang ------
---- mae ------
<tr> 
  <td> 1 </td> 
  <td> RDKit2D + MLP (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/mims-harvard/TDC/tree/master/examples/single_pred/admet"> GitHub</a>, <a href="https://doi.org/10.1093/bioinformatics/btaa1005">Paper </a></td> 
  <td> 633,409 </td> 
  <td>0.393 <span>&#177;</span> 0.024 </td> 
</tr>
<tr> 
  <td> 2 </td> 
  <td> CNN (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/mims-harvard/TDC/tree/master/examples/single_pred/admet"> GitHub</a>, <a href="https://doi.org/10.1093/bioinformatics/btaa1005">Paper </a></td> 
  <td> 226,625 </td> 
  <td>0.446 <span>&#177;</span> 0.036 </td> 
</tr>
<tr> 
  <td> 3 </td> 
  <td> Morgan + MLP (DeepPurpose) </td> 
  <td><a href="mailto:kexinhuang@hsph.harvard.edu">Kexin Huang</a></td> 
  <td><a href="https://github.com/mim

## Docking Group

In [6]:
df_docking = df[df.Group == 'Docking Group']

In [10]:
df_docking

,Time,Name,Email,Model,Paper_URL,Code_URL,Hardware,#Params,Group,Text_Result,PKL_Result,Result
4,2021/05/19 3:57:00 PM MDT,Tianfan Fu,tfu42@gatech.edu,GCPN,https://arxiv.org/abs/1806.02473,https://github.com/futianfan/GCPN,Titan GPU,"17,600",Docking Group,NaN,https://drive.google.com/u/0/open?usp=forms_we...,NaN
5,2021/05/19 3:58:05 PM MDT,Tianfan Fu,tfu42@gatech.edu,SMILES-LSTM,https://pubs.acs.org/doi/10.1021/acscentsci.7b...,https://github.com/futianfan/guacamol_tdc,GPU Titan,"3,149,000",Docking Group,NaN,https://drive.google.com/u/0/open?usp=forms_we...,NaN
6,2021/05/19 3:59:00 PM MDT,Tianfan Fu,tfu42@gatech.edu,Graph-GA,https://pubs.rsc.org/en/content/articlelanding...,https://github.com/futianfan/guacamol_tdc,CPU,0,Docking Group,NaN,https://drive.google.com/u/0/open?usp=forms_we...,NaN
7,2021/06/01 8:39:47 PM MDT,Tianfan Fu,tfu42@gatech.edu,MolDQN,https://www.nature.com/articles/s41598-019-471...,https://github.com/futianfan/mol_dqn_docking,GPUs,2694800,Docking Group,NaN,https://drive.google.com/u/0/open?usp=forms_we...,NaN
8,2021/06/01 8:41:21 PM MDT,Tianfan Fu,tfu42@gatech.edu,MARS,https://openreview.net/pdf?id=kHSu4ebxFXY,https://github.com/futianfan/MARS,GPUs,153000,Docking Group,NaN,https://drive.google.com/u/0/open?usp=forms_we...,NaN


In [7]:
from Google import Create_Service

CLIENT_SECRET_FILE = 'client_secret_file.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

client_secret_file.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']


RefreshError: ('invalid_grant: Token has been expired or revoked.', '{\n  "error": "invalid_grant",\n  "error_description": "Token has been expired or revoked."\n}')

In [8]:
import os, io
from googleapiclient.http import MediaIoBaseDownload

file_ids = [i.split('id=')[1] for i in df_docking.PKL_Result.values]
file_names = [i + '.pkl' for i in df_docking.Model.values]

for file_id, file_name in zip(file_ids, file_names):
    request = service.files().get_media(fileId = file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fd = fh, request = request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print('Download progress {0}'.format(status.progress() * 100))

    fh.seek(0)

    with open(os.path.join('./docking_result', file_name), 'wb') as f:
        f.write(fh.read())
        f.close()

Download progress 100.0
Download progress 100.0
Download progress 100.0


In [12]:
import pickle
import os, io

def get_result(x):
    with open(os.path.join('./docking_result', x + '.pkl'), 'rb') as f:      
        x = pickle.load(f)
    return x
              
df_docking['Result'] = df_docking.Model.apply(lambda x: get_result(x))

/Users/kexinhuang/opt/anaconda3/envs/DeepPurpose/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [14]:
out = {df_docking.Model.iloc[ind]: df_docking.Result.iloc[ind]['DRD3'] for ind in range(len(df_docking))}
out.keys()

dict_keys(['GCPN', 'SMILES-LSTM', 'Graph-GA', 'MolDQN', 'MARS'])

In [17]:
out = {df_docking.Model.iloc[ind]: df_docking.Result.iloc[ind]['DRD3'] for ind in range(len(df_docking))}

for call in [100, 500, 1000, 5000]:
    print(call)
    for x in ['top100', 'top10', 'top1', 'diversity', 'novelty', '%pass', 'top1_%pass', 'm1']:
        print("& {:.3f}\std{{{:.3f}}} & {:.3f}\std{{{:.3f}}} & {:.3f}\std{{{:.3f}}} & {:.3f}\std{{{:.3f}}} & {:.3f}\std{{{:.3f}}}".format(out['Graph-GA'][call][x][0], 
                                                                                            out['Graph-GA'][call][x][1],
                                                                                            out['SMILES-LSTM'][call][x][0], 
                                                                                            out['SMILES-LSTM'][call][x][1],
                                                                                            out['GCPN'][call][x][0], 
                                                                                            out['GCPN'][call][x][1],
                                                                                            out['MolDQN'][call][x][0], 
                                                                                            out['MolDQN'][call][x][1],
                                                                                            out['MARS'][call][x][0], 
                                                                                            out['MARS'][call][x][1]))

100
& -7.222\std{0.013} & -7.594\std{0.182} & 3.860\std{0.102} & -5.178\std{0.341} & -5.928\std{0.298}
& -10.177\std{0.158} & -10.033\std{0.186} & -5.617\std{0.413} & -6.438\std{0.176} & -8.133\std{0.328}
& -11.767\std{1.087} & -11.133\std{0.634} & -11.633\std{2.217} & -7.020\std{0.194} & -9.100\std{0.712}
& 0.885\std{0.001} & 0.884\std{0.002} & 0.909\std{0.001} & 0.907\std{0.001} & 0.873\std{0.010}
& 1.000\std{0.000} & 1.000\std{0.000} & 1.000\std{0.000} & 1.000\std{0.000} & 1.000\std{0.000}
& 0.693\std{0.037} & 0.763\std{0.019} & 0.093\std{0.009} & 0.017\std{0.012} & 0.807\std{0.033}
& 0.000\std{0.000} & -1.100\std{1.417} & 7.667\std{0.262} & -3.630\std{2.588} & -3.633\std{0.946}
& 5.223\std{0.256} & 5.219\std{0.247} & 10.000\std{0.000} & 10.000\std{0.000} & 4.470\std{1.047}
500
& -10.036\std{0.221} & -9.419\std{0.173} & -8.119\std{0.104} & -6.357\std{0.084} & -7.278\std{0.198}
& -11.527\std{0.533} & -10.687\std{0.335} & -10.230\std{0.354} & -7.173\std{0.166} & -9.067\std{0.377}
& -1

In [18]:
## generate molecule images
from rdkit import Chem
from rdkit.Chem import Draw

for submit_idx in range(len(df_docking)):
    temp = df_docking.iloc[submit_idx]

    for target in temp.Result.keys():
        for max_calls in temp.Result[target].keys():
            mols = [Chem.MolFromSmiles(i) for i in temp.Result[target][max_calls]['top smiles']]
            img = Draw.MolsToGridImage(mols, molsPerRow=6, subImgSize=(200, 200), maxMols=300)
            file_path = 'benchmark/docking_group/smiles/' + target.lower() + '/img/' + temp.Model + '_' + str(max_calls) + '.png'
            img.save(file_path)

In [11]:
## best in data
pred = dict(pd.read_csv('../tdc_leaderboard/docking_zinc_drd3_top100', sep = '\t', header = None).values)

mols = [Chem.MolFromSmiles(i) for i in list(pred.keys())]
img = Draw.MolsToGridImage(mols, molsPerRow=6, subImgSize=(200, 200), maxMols=300)
file_path = 'benchmark/docking_group/smiles/' + target.lower() + '/img/best_in_data.png'
img.save(file_path)

In [19]:
## generate html page for that image
html_code = ''
all_targets = list(temp.Result.keys())

for target in all_targets:    
    for submit_idx in range(len(df_docking)):
        temp = df_docking.iloc[submit_idx]
        html_code += '# Generated Molecules for '+ target + ' Docking Leaderboard - ' + temp.Model + ' \n'
        for max_calls in temp.Result[target].keys():
            html_code += '### ' + temp.Model + ' ' + str(max_calls) + ' Maximum Calls Top Generated Molecules\n'
            html_code += "\n"
            html_code += "<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/"+ target.lower() + "'> Go Back To Leaderboard</a> </b>"
            html_code += "\n"
            html_code += "<img src='/benchmark/docking_group/smiles/" + target.lower() + "/img/" + temp.Model + "_" + str(max_calls) + ".png'> \n"
            html_code += "\n"
            html_code += "\n"

In [20]:
print(html_code)

# Generated Molecules for DRD3 Docking Leaderboard - GCPN 
### GCPN 100 Maximum Calls Top Generated Molecules

<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/drd3'> Go Back To Leaderboard</a> </b>
<img src='/benchmark/docking_group/smiles/drd3/img/GCPN_100.png'> 


### GCPN 500 Maximum Calls Top Generated Molecules

<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/drd3'> Go Back To Leaderboard</a> </b>
<img src='/benchmark/docking_group/smiles/drd3/img/GCPN_500.png'> 


### GCPN 1000 Maximum Calls Top Generated Molecules

<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/drd3'> Go Back To Leaderboard</a> </b>
<img src='/benchmark/docking_group/smiles/drd3/img/GCPN_1000.png'> 


### GCPN 5000 Maximum Calls Top Generated Molecules

<b> <i class='fas fa-long-arrow-alt-left'></i> <a href='/benchmark/docking_group/drd3'> Go Back To Leaderboard</a> </b>
<img src='/benchmark/docking_group/smiles/d

In [21]:
def get_entries(bm, max_calls):
    results = df_docking.Result.apply(lambda x: x[bm][max_calls]['top100'][0]).values
    
    out = np.argsort(results)
    idx2rank = dict(zip(out, list(range(len(out)))))
    rank2idx = dict(zip(list(range(len(out))), out))
    
    print('---- ' + bm + ' ------')
    entries = []
    for submit_idx in range(len(df_docking)):
        temp = df_docking.iloc[submit_idx]
        html_code = '<tr> \n'
        html_code = html_code + '  <td> ' + str(idx2rank[submit_idx] + 1) + ' </td> \n'        
        html_code = html_code + '  <td> ' + temp.Model + ' </td> \n'
        html_code = html_code + '  <td><a href="mailto:' + temp.Email + '">' + temp.Name + '</a></td> \n'
        html_code = html_code + '  <td><a href="' + temp.Code_URL + '"> GitHub</a>, <a href="' + temp.Paper_URL + '">Paper </a></td> \n'
        value = temp['#Params']
        value = int(str(value).replace(',', ''))
        html_code = html_code + '  <td> ' + f'{value:,}' + ' </td> \n'
        for metric in ['top100', 'top10', 'top1', 'diversity', 'novelty', '%pass', 'top1_%pass', 'm1']:
            html_code = html_code + '  <td>' + format(temp.Result[bm][max_calls][metric][0], '.3f') + ' <span>&#177;</span> ' + format(temp.Result[bm][max_calls][metric][1], '.3f') +' </td> \n'
        
        html_code += "  <td> <a href='/benchmark/docking_group/smiles/" + bm.lower() + "/" + temp.Model.lower() + "_smiles/#" + temp.Model.lower() + "-" + str(max_calls) +"-maximum-calls-top-generated-molecules'> Link </a> </td> \n"
        html_code += '</tr>'

        entries.append(html_code)
    
    for i in range(len(out)):
        print(entries[rank2idx[i]])

In [26]:
get_entries('DRD3', 100)

---- DRD3 ------
<tr> 
  <td> 1 </td> 
  <td> SMILES-LSTM </td> 
  <td><a href="mailto:tfu42@gatech.edu">Tianfan Fu</a></td> 
  <td><a href="https://github.com/futianfan/guacamol_tdc"> GitHub</a>, <a href="https://pubs.acs.org/doi/10.1021/acscentsci.7b00512">Paper </a></td> 
  <td> 3,149,000 </td> 
  <td>-7.594 <span>&#177;</span> 0.182 </td> 
  <td>-10.033 <span>&#177;</span> 0.186 </td> 
  <td>-11.133 <span>&#177;</span> 0.634 </td> 
  <td>0.884 <span>&#177;</span> 0.002 </td> 
  <td>1.000 <span>&#177;</span> 0.000 </td> 
  <td>0.763 <span>&#177;</span> 0.019 </td> 
  <td>-1.100 <span>&#177;</span> 1.417 </td> 
  <td>5.219 <span>&#177;</span> 0.247 </td> 
  <td> <a href='/benchmark/docking_group/smiles/drd3/smiles-lstm_smiles/#smiles-lstm-100-maximum-calls-top-generated-molecules'> Link </a> </td> 
</tr>
<tr> 
  <td> 2 </td> 
  <td> Graph-GA </td> 
  <td><a href="mailto:tfu42@gatech.edu">Tianfan Fu</a></td> 
  <td><a href="https://github.com/futianfan/guacamol_tdc"> GitHub</a>, <a hr